In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
data=pd.read_csv('/kaggle/input/lmsys-chatbot-arena/train.csv')

In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.utils import to_categorical

# Load data

# Preprocess data
# Create labels
data['winner'] = data.apply(lambda row: 0 if row['winner_model_a'] == 1 else (1 if row['winner_model_b'] == 1 else 2), axis=1)

# Encode categorical features
label_encoder = LabelEncoder()
data['model_a'] = label_encoder.fit_transform(data['model_a'])
data['model_b'] = label_encoder.fit_transform(data['model_b'])

# Feature and target split
X = data[['model_a', 'model_b', 'prompt', 'response_a', 'response_b']]
y = data['winner']

# Text vectorization (simple approach: count the number of words)
X['prompt_len'] = X['prompt'].apply(lambda x: len(str(x).split()))
X['response_a_len'] = X['response_a'].apply(lambda x: len(str(x).split()))
X['response_b_len'] = X['response_b'].apply(lambda x: len(str(x).split()))
X = X.drop(['prompt', 'response_a', 'response_b'], axis=1)

# Normalize features
scaler = StandardScaler()
X = scaler.fit_transform(X)

# One-hot encode labels
y = to_categorical(y, num_classes=3)

# Split data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Build the model
model = Sequential()
model.add(Dense(128, input_dim=X_train.shape[1], activation='relu'))
model.add(Dropout(0.1))
model.add(Dense(32, activation='relu'))
model.add(Dropout(0.1))
model.add(Dense(16, activation='relu'))
model.add(Dense(3, activation='softmax'))

# Compile the model
model.compile(optimizer=Adam(learning_rate=0.001), loss='categorical_crossentropy', metrics=['accuracy'])

# Define early stopping
early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)

# Train the model
history = model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=100, batch_size=32, callbacks=[early_stopping])

# Evaluate the model
loss, accuracy = model.evaluate(X_test, y_test)
print(f"Test Accuracy: {accuracy * 100:.2f}%")

# Save the model
model.save('chatbot_preference_model.h5')

# Print validation loss during training
val_loss = history.history['val_loss']
print(f"Validation Loss: {val_loss[-1]:.4f}")


In [ ]:
import pandas as pd
import numpy as np
from tensorflow.keras.models import load_model
from sklearn.preprocessing import LabelEncoder, StandardScaler

# Load the test data
test = pd.read_csv("/kaggle/input/lmsys-chatbot-arena/test.csv")

# Add default values for 'model_a' and 'model_b' as placeholders
test['model_a'] = 'gpt-3.5-turbo-0613'  # Example default value
test['model_b'] = 'gpt-3.5-turbo-0613'  # Example default value

# Encode categorical features using the same label encoder used for training data
label_encoder = LabelEncoder()
test['model_a'] = label_encoder.fit_transform(test['model_a'])
test['model_b'] = label_encoder.fit_transform(test['model_b'])

# Text vectorization (simple approach: count the number of words)
test['prompt_len'] = test['prompt'].apply(lambda x: len(str(x).split()))
test['response_a_len'] = test['response_a'].apply(lambda x: len(str(x).split()))
test['response_b_len'] = test['response_b'].apply(lambda x: len(str(x).split()))

# Drop original text columns
X_test = test.drop(['prompt', 'response_a', 'response_b'], axis=1)

# Normalize features using the same scaler used for training data
# Assuming the scaler was fit during training and is now loaded
scaler = StandardScaler()
X_test_scaled = scaler.fit_transform(X_test[['model_a', 'model_b', 'prompt_len', 'response_a_len', 'response_b_len']])

# Load the trained model
model = load_model('chatbot_preference_model.h5')

# Make predictions
probs = model.predict(X_test_scaled)

# Prepare the submission file
submission = pd.DataFrame(probs, columns=['winner_model_a', 'winner_model_b', 'winner_tie'])
submission['id'] = test['id']

# Reorder columns to have 'id' first
submission = submission[['id', 'winner_model_a', 'winner_model_b', 'winner_tie']]

submission.to_csv('submission.csv', index=False)
print("Submission file generated successfully!")


In [ ]:
print(submission)